## **Turi Image Similarity**
Welcome to the image similarit model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

- Load images for 101 different categories from Caltech's website
- Build an image similarity model.
- Convert the model to CoreML format and save it to the Skafos framework.

The example is based on [Turi Create's Image Similarity example](https://apple.github.io/turicreate/docs/userguide/image_similarity/).

--- 

Execute each cell one-by-one, by selecting the cell and do one of the following:

- Clicking the "play" button at the top of this frame.
- Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

In [ ]:
# Import necessary libraries
import urllib
import tarfile

import coremltools 
from skafossdk import *
import turicreate as tc

In [ ]:
# Initialize Skafos
ska = Skafos()

### 1. **Load the data**
The training data loaded below comes compressed from the [Computer Vision dept. at Caltech](http://www.vision.caltech.edu/Image_Datasets/Caltech101/Caltech101.html).  The training data for this example are 9,145 images from 101 different categories.  First, we need to unzip and extract the images.  We then add row numbers to the data set such that we can reference images later when we need to query the model for similar images.

In [ ]:
# Specify where to pull images from and local path name
data_url = "http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz";
data_path = "101_ObjectCategories.tar.gz" # specify output path name

# Fetch training data from Caltech's website and extract the contents.
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path, "r:gz")
tar.extractall()
tar.close()

# Load the data into a Turi Create SFrame
reference_data  = tc.image_analysis.load_images('./101_ObjectCategories')
reference_data = reference_data.add_row_number() # add a row number to the data set

In [ ]:
reference_data.head()

### 2. **Build the model**
We use the `tc.image_similarity.create` function and specify the data. We also pass None to the label argument, which tells the function to use the row number of each image as the label. This becomes important when you want to query the model for similar images later on. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/api/generated/turicreate.image_similarity.create.html).


**Note: GPU support is coming soon. The code below has been commented out because it will take a long time (~1 day) to train with CPUs. Additionally, if you don't update the training dataset, it will re-create the baseline image similarity model from the integration guide on the Skafos dashboard.**

In [ ]:
# Train an image similarity model. With label=None, the model uses row numbers to identify images when the model is queried.
#model = tc.image_similarity.create(reference_data, label=None)

### 3. **Deliver the model**
Once your model has been created, it must be converted to CoreML and saved to the Skafos framework. Once saved, if you wish to push to your iOS devices, you can use the `.deliver()` method below. We've left that commented out for now.

In [ ]:
# Specify the CoreML model name
model_name = 'ImageSimilarity'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Use coremltools to convert model weights to half-precision.
# This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Save model asset to Skafos
ska.asset_manager.save(
    name=model_name,              # Name used to load or deliver asset, also used within the Swift SDK.
    files=coreml_model_name,      # File or list of files to bundle together as a versioned asset.
    tags=['latest'],              # User-defined tags to help distinguish your asset.
    access='public'               # Asset access- public/private.
)

In [ ]:
# Deliver asset to devices (push)
#ska.asset_manager.deliver(
#  name=model_name,                # Name used to load or deliver asset, also used within the Swift SDK.
#  tag='latest',                   # User-defined tags to help distinguish your asset.
#  dev=True                        # Push asset through Apple's APNS dev or prod server
#)